In [6]:
spark.stop()

In [1]:
# Notebook phân tích & xây dựng mapping cho dữ liệu bất động sản
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, to_timestamp, current_timestamp, lit, regexp_replace, trim,
    when, upper, lower, split, element_at, round as spark_round,
    avg, count, percentile_approx, stddev, min as spark_min, max as spark_max,
    udf, length, expr
)
from pyspark.sql.types import StringType, DoubleType, BooleanType
from pyspark.sql.window import Window

# Thêm thư mục gốc vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

# Tạo Spark Session
spark = SparkSession.builder \
    .appName("BatDongSan Mapping Analysis") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

print("Spark session created successfully")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/24 19:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session created successfully


In [2]:
# Hàm tiện ích phân tích
def get_date_format(date_obj=None):
    """Trả về ngày theo định dạng YYYY-MM-DD"""
    if date_obj is None:
        date_obj = datetime.now()
    return date_obj.strftime("%Y-%m-%d")

def log_dataframe_info(df, name="dataframe"):
    """In thông tin về DataFrame"""
    print(f"\n===== Thông tin về {name} =====")
    print(f"Số lượng bản ghi: {df.count()}")
    print(f"Schema:")
    df.printSchema()
    print("\nMẫu dữ liệu:")
    df.show(5, truncate=False)

    # Thống kê null values
    null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    print("\nSố lượng giá trị NULL trong từng cột:")
    null_counts.show(truncate=False)

def get_unique_values(df, column_name, limit=100):
    """Hiển thị các giá trị duy nhất của một cột và số lượng của chúng"""
    value_counts = df.groupBy(column_name).count().orderBy("count", ascending=False)
    print(f"\n===== Giá trị duy nhất của cột {column_name} =====")
    value_counts.show(limit, truncate=False)
    return value_counts

def plot_histogram(df, column_name, bins=20, title=None):
    """Vẽ biểu đồ histogram cho một cột số"""
    if not title:
        title = f"Histogram của {column_name}"
    
    # Chuyển cột thành pandas để vẽ biểu đồ
    data = df.select(column_name).na.drop().toPandas()
    plt.figure(figsize=(10, 6))
    plt.hist(data[column_name], bins=bins, alpha=0.7)
    plt.title(title)
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_boxplot(df, column_name, title=None):
    """Vẽ biểu đồ boxplot cho một cột số"""
    if not title:
        title = f"Boxplot của {column_name}"
    
    # Chuyển cột thành pandas để vẽ biểu đồ
    data = df.select(column_name).na.drop().toPandas()
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=data[column_name])
    plt.title(title)
    plt.xlabel(column_name)
    plt.grid(True, alpha=0.3)
    plt.show()

def detect_outliers(df, column_name, method='iqr', threshold=1.5):
    """Phát hiện outlier dùng phương pháp IQR"""
    
    result = {}
    
    # Tính toán các thống kê
    stats = df.select(
        spark_min(column_name).alias("min"),
        percentile_approx(column_name, 0.25).alias("q1"),
        percentile_approx(column_name, 0.5).alias("median"),
        percentile_approx(column_name, 0.75).alias("q3"),
        spark_max(column_name).alias("max"),
        avg(column_name).alias("mean"),
        stddev(column_name).alias("stddev"),
        count(column_name).alias("count")
    ).collect()[0]
    
    # In các thống kê cơ bản
    print(f"\n===== Thống kê cho cột {column_name} =====")
    print(f"Minimum: {stats['min']}")
    print(f"Q1 (25%): {stats['q1']}")
    print(f"Median: {stats['median']}")
    print(f"Q3 (75%): {stats['q3']}")
    print(f"Maximum: {stats['max']}")
    print(f"Mean: {stats['mean']}")
    print(f"Standard Deviation: {stats['stddev']}")
    print(f"Count: {stats['count']}")
    
    # Tính IQR
    iqr = stats['q3'] - stats['q1']
    
    # Xác định ngưỡng outlier
    lower_bound = stats['q1'] - threshold * iqr
    upper_bound = stats['q3'] + threshold * iqr
    
    print(f"IQR: {iqr}")
    print(f"Lower Bound: {lower_bound}")
    print(f"Upper Bound: {upper_bound}")
    
    # Đếm số lượng outlier
    outliers_count = df.filter(
        (col(column_name) < lower_bound) | 
        (col(column_name) > upper_bound)
    ).count()
    
    outliers_percentage = (outliers_count / stats['count']) * 100 if stats['count'] > 0 else 0
    
    print(f"Số lượng outlier: {outliers_count} ({outliers_percentage:.2f}%)")
    
    result["lower_bound"] = lower_bound
    result["upper_bound"] = upper_bound
    result["outliers_count"] = outliers_count
    result["outliers_percentage"] = outliers_percentage
    result["stats"] = stats
    
    return result

In [35]:
# Đọc dữ liệu
# Ưu tiên sử dụng file CSV trong thư mục tmp nếu có
csv_file = "/home/fer/data/real_estate_project/tmp/csv_files/bds_data_may2025.csv"
json_path = "hdfs://namenode:9000/data/realestate/raw/batdongsan/house/2025/05/*"

try:
    # Thử đọc từ file CSV local
    if os.path.exists(csv_file):
        df = spark.read.option("header", "true").csv(csv_file)
        print(f"Đã đọc dữ liệu từ CSV local: {csv_file}")
    else:
        # Nếu không có file CSV, đọc từ JSON trên HDFS
        df = spark.read.option("multiline", "false").json(json_path)
        print(f"Đã đọc dữ liệu JSON từ: {json_path}")
    
    # Kiểm tra dữ liệu đọc được
    log_dataframe_info(df, "raw_data")
    
except Exception as e:
    print(f"Lỗi khi đọc dữ liệu: {str(e)}")
    # Tạo DataFrame trống nếu có lỗi
    from pyspark.sql.types import StructType, StructField, StringType
    schema = StructType([
        StructField("url", StringType(), True),
        StructField("title", StringType(), True),
        StructField("price", StringType(), True),
        StructField("price_per_m2", StringType(), True),
        StructField("area", StringType(), True),
        StructField("bedroom", StringType(), True),
        StructField("bathroom", StringType(), True),
        StructField("floor_count", StringType(), True),
        StructField("facade_width", StringType(), True),
        StructField("road_width", StringType(), True),
        StructField("house_direction", StringType(), True),
        StructField("legal_status", StringType(), True),
        StructField("interior", StringType(), True),
        StructField("location", StringType(), True),
        StructField("description", StringType(), True),
        StructField("posted_date", StringType(), True),
        StructField("crawl_timestamp", StringType(), True),
        StructField("latitude", StringType(), True),
        StructField("longitude", StringType(), True),
        StructField("seller_info", StringType(), True),
        StructField("source", StringType(), True),
        StructField("data_type", StringType(), True),
    ])
    df = spark.createDataFrame([], schema)

Đã đọc dữ liệu JSON từ: hdfs://namenode:9000/data/realestate/raw/batdongsan/house/2025/05/*

===== Thông tin về raw_data =====
Số lượng bản ghi: 17336
Schema:
root
 |-- area: string (nullable = true)
 |-- bathroom: string (nullable = true)
 |-- bedroom: string (nullable = true)
 |-- crawl_timestamp: string (nullable = true)
 |-- data_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- facade_width: string (nullable = true)
 |-- floor_count: string (nullable = true)
 |-- house_direction: string (nullable = true)
 |-- interior: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- legal_status: string (nullable = true)
 |-- location: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- posted_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- price_per_m2: string (nullable = true)
 |-- road_width: string (nullable = true)
 |-- seller_info: string (nullable = true)
 |-- source: string (nullable = true)
 |--

In [6]:
# 1. Phân tích và xây dựng mapping cho house_direction với tiếng Việt có dấu
print("\n===== PHÂN TÍCH HƯỚNG NHÀ (house_direction) =====")

# Hàm chuẩn hóa chuỗi: lowercase, loại bỏ dấu cách và dấu gạch ngang
def normalize_direction(value):
    if value is None:
        return None
    return value.lower().replace(" ", "").replace("-", "")

# Tạo UDF để chuẩn hóa
normalize_direction_udf = udf(normalize_direction, StringType())

# Làm sạch và chuẩn hóa dữ liệu house_direction
cleaned_df = df.withColumn(
    "house_direction_normalized",
    normalize_direction_udf(col("house_direction"))
)

# Phân tích giá trị duy nhất sau khi chuẩn hóa
direction_counts = get_unique_values(cleaned_df, "house_direction_normalized")

# Tạo mapping đầy đủ với tất cả các biến thể có thể (có dấu và không dấu)
print("\n===== ĐỀ XUẤT MAPPING CHO HƯỚNG NHÀ =====")
direction_mapping = {
    # Hướng đơn - không dấu
    "dong": "EAST",
    "tay": "WEST", 
    "nam": "SOUTH",
    "bac": "NORTH",
    
    # Hướng đơn - có dấu
    "đông": "EAST",
    "tây": "WEST", 
    "nam": "SOUTH",
    "bắc": "NORTH",
    
    # Hướng kép - không dấu
    "dongnam": "SOUTHEAST",
    "dongbac": "NORTHEAST",
    "taynam": "SOUTHWEST",
    "taybac": "NORTHWEST",
    
    # Hướng kép - có dấu
    "đôngnam": "SOUTHEAST",
    "đôngbắc": "NORTHEAST", 
    "tâynam": "SOUTHWEST",
    "tâybắc": "NORTHWEST",
    
    # Các biến thể viết khác - không dấu
    "namdong": "SOUTHEAST",
    "bacdong": "NORTHEAST", 
    "namtay": "SOUTHWEST",
    "bactay": "NORTHWEST",
    
    # Các biến thể viết khác - có dấu
    "namđông": "SOUTHEAST",
    "bắcđông": "NORTHEAST", 
    "namtây": "SOUTHWEST",
    "bắctây": "NORTHWEST",
    
    # Trường hợp null hoặc không xác định
    None: "UNKNOWN",
    "": "UNKNOWN"
}

print("Mapping được tạo:")
for key, value in direction_mapping.items():
    print(f"  '{key}' -> '{value}'")

# Tạo UDF để áp dụng mapping
map_direction_udf = udf(lambda x: direction_mapping.get(x, "UNKNOWN"), StringType())

# Thử nghiệm mapping
mapped_directions = cleaned_df.withColumn(
    "house_direction_mapped", 
    map_direction_udf(col("house_direction_normalized"))
)

# Hiển thị kết quả mapping
print("\n===== KẾT QUẢ MAPPING =====")
get_unique_values(mapped_directions, "house_direction_mapped")

# Kiểm tra các giá trị chưa được map (UNKNOWN)
unknown_directions = mapped_directions.filter(col("house_direction_mapped") == "UNKNOWN")
if unknown_directions.count() > 0:
    print("\n===== CÁC GIÁ TRỊ CHƯA ĐƯỢC MAP =====")
    get_unique_values(unknown_directions, "house_direction_normalized")
    print("Cần bổ sung mapping cho các giá trị này!")


===== PHÂN TÍCH HƯỚNG NHÀ (house_direction) =====

===== Giá trị duy nhất của cột house_direction_normalized =====
+--------------------------+-----+
|house_direction_normalized|count|
+--------------------------+-----+
|                          |13729|
|đôngnam                   |668  |
|đôngbắc                   |515  |
|tâybắc                    |455  |
|đông                      |443  |
|tâynam                    |441  |
|nam                       |428  |
|bắc                       |350  |
|tây                       |307  |
+--------------------------+-----+


===== ĐỀ XUẤT MAPPING CHO HƯỚNG NHÀ =====
Mapping được tạo:
  'dong' -> 'EAST'
  'tay' -> 'WEST'
  'nam' -> 'SOUTH'
  'bac' -> 'NORTH'
  'đông' -> 'EAST'
  'tây' -> 'WEST'
  'bắc' -> 'NORTH'
  'dongnam' -> 'SOUTHEAST'
  'dongbac' -> 'NORTHEAST'
  'taynam' -> 'SOUTHWEST'
  'taybac' -> 'NORTHWEST'
  'đôngnam' -> 'SOUTHEAST'
  'đôngbắc' -> 'NORTHEAST'
  'tâynam' -> 'SOUTHWEST'
  'tâybắc' -> 'NORTHWEST'
  'namdong' -> 'SOUTHEAS

In [8]:
# 2. Phân tích và xây dựng mapping cho interior với keyword matching
print("\n===== PHÂN TÍCH NỘI THẤT (interior) VỚI KEYWORD MATCHING =====")

# Hàm chuẩn hóa cho interior (giữ nguyên để phân tích)
def normalize_interior(value):
    if value is None:
        return None
    normalized = value.lower().replace(" ", "").replace("-", "").replace(".", "").replace(",", "")
    return normalized

# Hàm mapping theo keyword
def map_interior_by_keywords(value):
    if value is None or value == "":
        return "UNKNOWN"
    
    value_lower = value.lower()
    
    # Kiểm tra các keyword cho LUXURY (ưu tiên cao nhất)
    luxury_keywords = [
        "caocấp", "cao cấp", "luxury", "sangtrọng", "sang trọng", "xịn", "5*", "5 sao",
        "nhậpkhẩu", "nhập khẩu", "châuâu", "châu âu", "tiêuchuẩn", "tiêu chuẩn",
        "chuẩnkhách", "chuẩn khách", "hạngsang", "hạng sang"
    ]
    
    for keyword in luxury_keywords:
        if keyword in value_lower:
            return "LUXURY"
    
    # Kiểm tra các keyword cho FULLY_FURNISHED
    fully_furnished_keywords = [
        "đầyđủ", "đầy đủ", "full", "hoànthiện", "hoàn thiện",
        "trangbị", "trang bị", "điềuhòa", "điều hòa", "tủlạnh", "tủ lạnh",
        "nộithất", "nội thất", "đểlại", "để lại", "tặng"
    ]
    
    for keyword in fully_furnished_keywords:
        if keyword in value_lower:
            return "FULLY_FURNISHED"
    
    # Kiểm tra các keyword cho BASIC
    basic_keywords = [
        "cơbản", "cơ bản", "bìnhthường", "bình thường", "chuẩn"
    ]
    
    for keyword in basic_keywords:
        if keyword in value_lower:
            return "BASIC"
    
    # Kiểm tra các keyword cho UNFURNISHED
    unfurnished_keywords = [
        "thô", "trống", "không", "k ", "nt", "nhàthô", "nhà thô"
    ]
    
    for keyword in unfurnished_keywords:
        if keyword in value_lower:
            return "UNFURNISHED"
    
    return "UNKNOWN"

# Tạo UDF để áp dụng keyword mapping
from pyspark.sql.types import StringType
map_interior_keywords_udf = udf(map_interior_by_keywords, StringType())

# Áp dụng mapping mới
mapped_interiors_keywords = cleaned_df.withColumn(
    "interior_mapped_keywords", 
    map_interior_keywords_udf(col("interior"))
)

print("\n===== KẾT QUẢ MAPPING VỚI KEYWORD MATCHING =====")
get_unique_values(mapped_interiors_keywords, "interior_mapped_keywords")

# So sánh với mapping cũ
print("\n===== SO SÁNH KẾT QUẢ =====")
comparison_df = mapped_interiors_keywords.groupBy("interior_mapped_keywords").count().orderBy("count", ascending=False)
comparison_df.show()

# Kiểm tra các giá trị vẫn còn UNKNOWN
unknown_interiors_keywords = mapped_interiors_keywords.filter(col("interior_mapped_keywords") == "UNKNOWN")
if unknown_interiors_keywords.count() > 0:
    print(f"\nSố lượng giá trị vẫn UNKNOWN: {unknown_interiors_keywords.count()}")
    print("\n===== MỘT SỐ GIÁ TRỊ VẪN UNKNOWN =====")
    unknown_samples = unknown_interiors_keywords.select("interior").distinct().limit(20)
    unknown_samples.show(truncate=False)


===== PHÂN TÍCH NỘI THẤT (interior) VỚI KEYWORD MATCHING =====

===== KẾT QUẢ MAPPING VỚI KEYWORD MATCHING =====

===== Giá trị duy nhất của cột interior_mapped_keywords =====
+------------------------+-----+
|interior_mapped_keywords|count|
+------------------------+-----+
|UNKNOWN                 |9954 |
|FULLY_FURNISHED         |5336 |
|BASIC                   |1854 |
|LUXURY                  |172  |
|UNFURNISHED             |20   |
+------------------------+-----+


===== SO SÁNH KẾT QUẢ =====
+------------------------+-----+
|interior_mapped_keywords|count|
+------------------------+-----+
|                 UNKNOWN| 9954|
|         FULLY_FURNISHED| 5336|
|                   BASIC| 1854|
|                  LUXURY|  172|
|             UNFURNISHED|   20|
+------------------------+-----+


Số lượng giá trị vẫn UNKNOWN: 9954

===== MỘT SỐ GIÁ TRỊ VẪN UNKNOWN =====
+---------------------------+
|interior                   |
+---------------------------+
|24/5                       |
|T

In [22]:
# Cải thiện mapping cho legal_status với phân loại chi tiết hơn
def map_legal_status_by_keywords_v2(value):
    if value is None or value == "":
        return "UNKNOWN"
    
    # Normalize: lowercase, loại bỏ tất cả dấu cách và ký tự đặc biệt
    value_lower = (
        value.lower()
        .replace(" ", "")
        .replace("-", "")
        .replace(".", "")
        .replace("/", "")
        .replace("\\", "")
        .replace(",", "")
        .replace(":", "")
        .replace(";", "")
        .replace("(", "")
        .replace(")", "")
        .replace("+", "")
    )
    
    # NO_LEGAL - Không pháp lý hoặc không sổ (kiểm tra đầu tiên)
    no_legal_keywords = [
        "khôngpháplý", "khongphapły", "khôngsổ", "khongso", "kosổ", "koso",
        "không", "khong", "chưacó", "chuaco", "chưa", "chua", "ko"
    ]
    if any(keyword in value_lower for keyword in no_legal_keywords):
        return "NO_LEGAL"

    # LAND_USE_CERTIFICATE - Thổ cư riêng biệt
    land_use_keywords = [
        "thổcư", "thocu", "thổcư100", "thocu100", "thổcư100%", "thocu100%",
        "đấtthổcư", "datthoju", "cnqsdđ", "cnqsdd", "sửdụngđất", "sudungdat"
    ]
    if any(keyword in value_lower for keyword in land_use_keywords):
        return "LAND_USE_CERTIFICATE"

    # RED_BOOK - Sổ đỏ/hồng (thu hẹp lại, chỉ những trường hợp rõ ràng)
    red_book_keywords = [
        # Sổ đỏ/hồng cơ bản - chính xác
        "sổđỏ", "sodo", "sổhồng", "sohong", "sổđỏsổhồng", "sđcc", "sdhh",
        
        # Các biến thể rõ ràng của sổ đỏ/hồng
        "bìađỏ", "biado", "sổchínhchủ", "sochinhchu", 
        
        # Chỉ những trường hợp thực sự có sổ đỏ/hồng
        "sổđẹp", "sodep", "sổvuông", "sovuong", "sổvuôngvắn", "sovuongvan",
        "sổsạch", "sosach", "sổđ", "sod"
    ]
    if any(keyword in value_lower for keyword in red_book_keywords):
        return "RED_BOOK"

    # OWNERSHIP_CERTIFICATE - Chứng nhận quyền sở hữu và các loại sổ khác
    ownership_keywords = [
        # Các loại chứng nhận
        "shcc", "shr", "ccqsh", "chứngnhận", "chungnhan", "sổcc", "socc",
        "côngcông", "congcong", "sổcôngnhận", "socognhan", "sổcôngnhậnđủ", "socognhanđu",
        "sổhoàncông", "sohoancong", "vbcn", "giấychứngnhận", "giaychungnhan",
        
        # Pháp lý rõ ràng nhưng không chỉ rõ loại sổ
        "pháplý", "phapły", "pháplýsạch", "phaplysach", "pháplýchuẩn", "phaplychuan",
        "pháplýcánhân", "phaplycanhân", "pháplýrõràng", "phaplýrorang",
        
        # Quyền sở hữu
        "chínhchủ", "chinhchu", "sởhữu", "sohuu", "quyềnsởhữu", "quyensohuu", 
        "chủquyền", "chuquyen", "đồngsởhữu", "dongsohuu",
        
        # Có giấy tờ nhưng không rõ loại
        "có", "co", "cósố", "coso", "đầyđủ", "daydu", "sổsẵn", "sosan",
        "giấytờ", "giayto", "giấytờđầyđủ", "giaytodaydu"
    ]
    if any(keyword in value_lower for keyword in ownership_keywords):
        return "OWNERSHIP_CERTIFICATE"

    # TRANSACTION_READY - Sẵn sàng giao dịch nhưng không rõ loại giấy tờ
    transaction_keywords = [
        "sẵnsànggiaodịch", "sansanggiaodich", "côngchứng", "congchung",
        "giaodịchngay", "giaodichngay", "vuôngvắn", "vuongvan", "sẵnsàn", "sansan",
        "côngchứngtrongnày", "congchungtrongngay", "giaodịchnhanh", "giaodichhanh",
        "sạch", "sach", "chuẩn", "chuan", "rõràng", "rorang", "thiệnchí", "thienchi"
    ]
    if any(keyword in value_lower for keyword in transaction_keywords):
        return "TRANSACTION_READY"

    # PURCHASE_CONTRACT - Hợp đồng mua bán và giấy tờ tay
    contract_keywords = [
        "hđmb", "hdmb", "hợpđồng", "hopdong", "hợpđồngmuabán", "hopdongmuaban",
        "viếtsổ", "vietso", "giấytay", "giaytay", "ccvb", "vibằng", "vibang",
        "hđ", "hd"
    ]
    if any(keyword in value_lower for keyword in contract_keywords):
        return "PURCHASE_CONTRACT"

    # PENDING_CERTIFICATE - Đang chờ sổ hoặc sắp có sổ
    pending_keywords = [
        "đangchờ", "dangcho", "chờsổ", "choso", "sắpcấp", "sapcap",
        "đangxử", "dangxu", "chờcấp", "chocap", "sắpcó", "sapco",
        "chờra", "chora", "đanghoànthành", "danghoanthanh", "làmsổ", "lamso",
        "nhậnnhàvàlàmsổ", "nhanhanvalamso", "sổgửibank", "soguibank"
    ]
    if any(keyword in value_lower for keyword in pending_keywords):
        return "PENDING_CERTIFICATE"

    # INDIVIDUAL_CERTIFICATE - Sổ riêng và cá nhân
    individual_keywords = [
        "sổriêng", "sorieng", "riêng", "rieng", "tách", "tach",
        "độclập", "doclap", "cánhân", "canhan", "cá", "ca", "nhân", "nhan",
        "thừakế", "thuake"
    ]
    if any(keyword in value_lower for keyword in individual_keywords):
        return "INDIVIDUAL_CERTIFICATE"

    return "UNKNOWN"

# Áp dụng mapping v2
print("\n===== ÁP DỤNG MAPPING V2 CHO LEGAL_STATUS =====")
map_legal_status_v2_udf = udf(map_legal_status_by_keywords_v2, StringType())

mapped_legal_status_v2 = cleaned_df.withColumn(
    "legal_status_mapped_v2", 
    map_legal_status_v2_udf(col("legal_status"))
)

print("\n===== KẾT QUẢ MAPPING V2 =====")
get_unique_values(mapped_legal_status_v2, "legal_status_mapped_v2")

# So sánh v1 vs v2
comparison_v1_v2 = mapped_legal_status_v2.withColumn(
    "legal_status_mapped_v1", 
    map_legal_status_improved_udf(col("legal_status"))
)

print("\n===== SO SÁNH V1 VS V2 =====")
comparison_v1v2 = comparison_v1_v2.groupBy("legal_status_mapped_v1", "legal_status_mapped_v2").count().orderBy("count", ascending=False)
comparison_v1v2.show(20, truncate=False)


===== ÁP DỤNG MAPPING V2 CHO LEGAL_STATUS =====

===== KẾT QUẢ MAPPING V2 =====

===== Giá trị duy nhất của cột legal_status_mapped_v2 =====
+----------------------+-----+
|legal_status_mapped_v2|count|
+----------------------+-----+
|RED_BOOK              |13852|
|UNKNOWN               |3245 |
|OWNERSHIP_CERTIFICATE |112  |
|PURCHASE_CONTRACT     |69   |
|PENDING_CERTIFICATE   |27   |
|NO_LEGAL              |20   |
|INDIVIDUAL_CERTIFICATE|8    |
|LAND_USE_CERTIFICATE  |2    |
|TRANSACTION_READY     |1    |
+----------------------+-----+


===== SO SÁNH V1 VS V2 =====
+----------------------+----------------------+-----+
|legal_status_mapped_v1|legal_status_mapped_v2|count|
+----------------------+----------------------+-----+
|RED_BOOK              |RED_BOOK              |13850|
|UNKNOWN               |UNKNOWN               |3243 |
|RED_BOOK              |OWNERSHIP_CERTIFICATE |110  |
|PURCHASE_CONTRACT     |PURCHASE_CONTRACT     |69   |
|PENDING_CERTIFICATE   |PENDING_CERTIFICATE   

In [23]:
# 3. Phân tích và làm sạch dữ liệu số (area, price, bedroom, bathroom)
print("\n===== PHÂN TÍCH VÀ LÀM SẠCH DỮ LIỆU SỐ =====")

# Chuyển đổi các cột số
def clean_numeric_data(df):
    """Làm sạch và chuyển đổi dữ liệu số"""
    
    # Làm sạch area
    df_cleaned = df.withColumn(
        "area_cleaned", 
        regexp_replace(col("area"), "[^0-9\\.]", "").cast("double")
    )
    
    # Làm sạch bedroom
    df_cleaned = df_cleaned.withColumn(
        "bedroom_cleaned", 
        regexp_replace(col("bedroom"), "[^0-9]", "").cast("double")
    )
    
    # Làm sạch bathroom
    df_cleaned = df_cleaned.withColumn(
        "bathroom_cleaned", 
        regexp_replace(col("bathroom"), "[^0-9]", "").cast("double")
    )
    
    # Làm sạch floor_count
    df_cleaned = df_cleaned.withColumn(
        "floor_count_cleaned", 
        regexp_replace(col("floor_count"), "[^0-9]", "").cast("double")
    )
    
    # Làm sạch facade_width
    df_cleaned = df_cleaned.withColumn(
        "facade_width_cleaned", 
        regexp_replace(col("facade_width"), "[^0-9\\.]", "").cast("double")
    )
    
    # Làm sạch road_width
    df_cleaned = df_cleaned.withColumn(
        "road_width_cleaned", 
        regexp_replace(col("road_width"), "[^0-9\\.]", "").cast("double")
    )
    
    return df_cleaned

# Áp dụng làm sạch
numeric_cleaned_df = clean_numeric_data(mapped_legal_status_v2)

# Phân tích area
print("\n===== PHÂN TÍCH DIỆN TÍCH (AREA) =====")
area_stats = detect_outliers(numeric_cleaned_df, "area_cleaned")

# Lọc area hợp lý (10m² đến 1000m²)
reasonable_area_df = numeric_cleaned_df.filter(
    (col("area_cleaned") >= 10) & (col("area_cleaned") <= 1000)
)
print(f"Số bản ghi sau khi lọc area hợp lý: {reasonable_area_df.count()}")

# Phân tích bedroom
print("\n===== PHÂN TÍCH SỐ PHÒNG NGỦ (BEDROOM) =====")
bedroom_stats = get_unique_values(reasonable_area_df, "bedroom_cleaned", 20)

# Lọc bedroom hợp lý (1-10 phòng)
reasonable_bedroom_df = reasonable_area_df.filter(
    (col("bedroom_cleaned") >= 1) & (col("bedroom_cleaned") <= 10)
)
print(f"Số bản ghi sau khi lọc bedroom hợp lý: {reasonable_bedroom_df.count()}")

# Phân tích bathroom
print("\n===== PHÂN TÍCH SỐ PHÒNG TẮM (BATHROOM) =====")
bathroom_stats = get_unique_values(reasonable_bedroom_df, "bathroom_cleaned", 15)

# Lọc bathroom hợp lý (1-5 phòng)
reasonable_bathroom_df = reasonable_bedroom_df.filter(
    (col("bathroom_cleaned") >= 1) & (col("bathroom_cleaned") <= 5)
)
print(f"Số bản ghi sau khi lọc bathroom hợp lý: {reasonable_bathroom_df.count()}")


===== PHÂN TÍCH VÀ LÀM SẠCH DỮ LIỆU SỐ =====

===== PHÂN TÍCH DIỆN TÍCH (AREA) =====

===== Thống kê cho cột area_cleaned =====
Minimum: 1.0
Q1 (25%): 80.0
Median: 126.0
Q3 (75%): 250.0
Maximum: 66171.0
Mean: 406.6580533289086
Standard Deviation: 1877.6895574470827
Count: 17327
IQR: 170.0
Lower Bound: -175.0
Upper Bound: 505.0
Số lượng outlier: 2184 (12.60%)
Số bản ghi sau khi lọc area hợp lý: 16076

===== PHÂN TÍCH SỐ PHÒNG NGỦ (BEDROOM) =====

===== Giá trị duy nhất của cột bedroom_cleaned =====
+---------------+-----+
|bedroom_cleaned|count|
+---------------+-----+
|null           |7667 |
|4.0            |1874 |
|3.0            |1416 |
|5.0            |1085 |
|2.0            |963  |
|6.0            |777  |
|1.0            |424  |
|8.0            |349  |
|7.0            |283  |
|10.0           |246  |
|9.0            |177  |
|12.0           |109  |
|20.0           |65   |
|11.0           |55   |
|14.0           |52   |
|15.0           |51   |
|16.0           |39   |
|13.0           

In [27]:
# 4. Xử lý và phân tích giá (price và price_per_m2)
print("\n===== XỬ LÝ VÀ PHÂN TÍCH GIÁ =====")

def clean_price_data(df):
    """Làm sạch và chuyển đổi dữ liệu giá"""
    
    # Xử lý trường giá với thỏa thuận
    df_price = df.withColumn("price_text", trim(col("price")))
    
    # Đánh dấu thỏa thuận
    df_price = df_price.withColumn(
        "is_negotiable", 
        when(
            lower(col("price_text")).contains("thỏa thuận") |
            lower(col("price_text")).contains("thoathuan") |
            lower(col("price_text")).contains("thoa thuan"), 
            lit(True)
        ).otherwise(lit(False))
    )
    
    # Chuyển đổi giá
    df_price = df_price.withColumn(
        "price_cleaned",
        when(
            col("is_negotiable"), 
            lit(None)
        )
        .when(
            lower(col("price_text")).contains("tỷ") |
            lower(col("price_text")).contains("ty"),
            regexp_replace(col("price_text"), "[^0-9\\.]", "").cast("double") * 1000000000
        )
        .when(
            lower(col("price_text")).contains("triệu") |
            lower(col("price_text")).contains("trieu"),
            regexp_replace(col("price_text"), "[^0-9\\.]", "").cast("double") * 1000000
        )
        .otherwise(
            regexp_replace(col("price_text"), "[^0-9\\.]", "").cast("double")
        )
    )
    
    # Xử lý price_per_m2
    df_price = df_price.withColumn("price_per_m2_text", trim(col("price_per_m2")))
    
    df_price = df_price.withColumn(
        "price_per_m2_cleaned",
        when(
            lower(col("price_per_m2_text")).contains("tỷ") |
            lower(col("price_per_m2_text")).contains("ty"),
            regexp_replace(col("price_per_m2_text"), "[^0-9\\.]", "").cast("double") * 1000000000
        )
        .when(
            lower(col("price_per_m2_text")).contains("triệu") |
            lower(col("price_per_m2_text")).contains("trieu"),
            regexp_replace(col("price_per_m2_text"), "[^0-9\\.]", "").cast("double") * 1000000
        )
        .otherwise(
            regexp_replace(col("price_per_m2_text"), "[^0-9\\.]", "").cast("double")
        )
    )
    
    return df_price.drop("price_text", "price_per_m2_text")

# Áp dụng làm sạch giá
price_cleaned_df = clean_price_data(df)

# Phân tích giá
print("\n===== PHÂN TÍCH GIÁ (PRICE) =====")
price_with_value = price_cleaned_df.filter(col("price_cleaned").isNotNull())
print(f"Số bản ghi có giá: {price_with_value.count()}")

if price_with_value.count() > 0:
    price_stats = detect_outliers(price_with_value, "price_cleaned")
    
    # Lọc giá hợp lý (100 triệu đến 100 tỷ)
    reasonable_price_df = price_with_value.filter(
        (col("price_cleaned") >= 100000000) & (col("price_cleaned") <= 100000000000)
    )
    print(f"Số bản ghi sau khi lọc giá hợp lý: {reasonable_price_df.count()}")

# Phân tích thỏa thuận
negotiable_count = price_cleaned_df.filter(col("is_negotiable") == True).count()
total_count = price_cleaned_df.count()
negotiable_rate = (negotiable_count / total_count * 100) if total_count > 0 else 0
print(f"\nTỷ lệ tin 'thỏa thuận': {negotiable_rate:.1f}% ({negotiable_count}/{total_count})")

# Phân tích price_per_m2
print("\n===== PHÂN TÍCH GIÁ/M² (PRICE_PER_M2) =====")
price_per_m2_with_value = price_cleaned_df.filter(col("price_per_m2_cleaned").isNotNull())
print(f"Số bản ghi có giá/m²: {price_per_m2_with_value.count()}")

if price_per_m2_with_value.count() > 0:
    price_per_m2_stats = detect_outliers(price_per_m2_with_value, "price_per_m2_cleaned")


===== XỬ LÝ VÀ PHÂN TÍCH GIÁ =====

===== PHÂN TÍCH GIÁ (PRICE) =====
Số bản ghi có giá: 16559

===== Thống kê cho cột price_cleaned =====
Minimum: 15000000.0
Q1 (25%): 35000000000.0
Median: 78000000000.0
Q3 (75%): 161000000000.0
Maximum: 15999000000000.0
Mean: 156890683918.111
Standard Deviation: 333648160427.5351
Count: 16559
IQR: 126000000000.0
Lower Bound: -154000000000.0
Upper Bound: 350000000000.0
Số lượng outlier: 1454 (8.78%)
Số bản ghi sau khi lọc giá hợp lý: 9761

Tỷ lệ tin 'thỏa thuận': 4.4% (768/17336)

===== PHÂN TÍCH GIÁ/M² (PRICE_PER_M2) =====
Số bản ghi có giá/m²: 16559

===== Thống kê cho cột price_per_m2_cleaned =====
Minimum: 108.0
Q1 (25%): 3125000000.0
Median: 15972000000.0
Q3 (75%): 32857000000.0
Maximum: 92491000000000.0
Mean: 60773573896.30105
Standard Deviation: 986915632178.8951
Count: 16559
IQR: 29732000000.0
Lower Bound: -41473000000.0
Upper Bound: 77455000000.0
Số lượng outlier: 655 (3.96%)


In [31]:
# Bổ sung import thiếu
from pyspark.sql.functions import (
    col, to_timestamp, current_timestamp, lit, regexp_replace, trim,
    when, upper, lower, split, element_at, round as spark_round,
    avg, count, percentile_approx, stddev, min as spark_min, max as spark_max,
    udf, length, expr, regexp_extract  # Thêm regexp_extract
)

# 6. Phân tích vị trí và coordinates (Sửa lại)
print("\n===== PHÂN TÍCH VỊ TRÍ VÀ COORDINATES =====")

# Làm sạch coordinates
coords_cleaned_df = df.withColumn(
    "latitude_cleaned", col("latitude").cast("double")
).withColumn(
    "longitude_cleaned", col("longitude").cast("double")
)

# Phân tích coordinates
valid_coords = coords_cleaned_df.filter(
    col("latitude_cleaned").isNotNull() & 
    col("longitude_cleaned").isNotNull() &
    (col("latitude_cleaned") >= 8.0) & (col("latitude_cleaned") <= 23.5) &  # Việt Nam lat range
    (col("longitude_cleaned") >= 102.0) & (col("longitude_cleaned") <= 110.0)  # Việt Nam lon range
)

print(f"Số bản ghi có coordinates hợp lệ: {valid_coords.count()}")
print(f"Tỷ lệ có coordinates: {(valid_coords.count() / coords_cleaned_df.count() * 100):.1f}%")

# Phân tích location text
print("\n===== PHÂN TÍCH LOCATION TEXT =====")
location_analysis = coords_cleaned_df.withColumn(
    "location_cleaned", trim(col("location"))
).withColumn(
    "location_length", length(col("location"))
)

# Thống kê độ dài location
location_stats = location_analysis.select(
    avg("location_length").alias("avg_length"),
    spark_min("location_length").alias("min_length"),
    spark_max("location_length").alias("max_length"),
    count(when(col("location_length") > 10, True)).alias("detailed_locations")
).collect()[0]

print(f"Độ dài location trung bình: {location_stats['avg_length']:.1f} ký tự")
print(f"Độ dài tối thiểu: {location_stats['min_length']}")
print(f"Độ dài tối đa: {location_stats['max_length']}")
print(f"Số location chi tiết (>10 ký tự): {location_stats['detailed_locations']}")

# Trích xuất thông tin từ location (Sửa lại)
def extract_location_info(df):
    """Trích xuất thông tin quận/huyện, thành phố từ location"""
    
    # Trích xuất quận/huyện với pattern cải thiện
    df_location = df.withColumn(
        "district_extracted",
        when(
            lower(col("location_cleaned")).contains("quận"),
            regexp_extract(lower(col("location_cleaned")), r"quận\s*(\d+)", 1)
        ).when(
            lower(col("location_cleaned")).contains("huyện"),
            regexp_extract(lower(col("location_cleaned")), r"huyện\s*([^,\s]+)", 1)
        ).when(
            lower(col("location_cleaned")).contains("q."),
            regexp_extract(lower(col("location_cleaned")), r"q\.?\s*(\d+)", 1)
        ).when(
            lower(col("location_cleaned")).contains("quận"),
            regexp_extract(lower(col("location_cleaned")), r"quận\s*([^,\s]+)", 1)
        ).otherwise(lit(None))
    )
    
    # Trích xuất thành phố với nhiều pattern hơn
    df_location = df_location.withColumn(
        "city_extracted",
        when(
            lower(col("location_cleaned")).contains("hồ chí minh") |
            lower(col("location_cleaned")).contains("tp.hcm") |
            lower(col("location_cleaned")).contains("tphcm") |
            lower(col("location_cleaned")).contains("hcm") |
            lower(col("location_cleaned")).contains("sài gòn") |
            lower(col("location_cleaned")).contains("saigon"),
            lit("Ho Chi Minh")
        ).when(
            lower(col("location_cleaned")).contains("hà nội") |
            lower(col("location_cleaned")).contains("hanoi") |
            lower(col("location_cleaned")).contains("ha noi"),
            lit("Hanoi")
        ).when(
            lower(col("location_cleaned")).contains("đà nẵng") |
            lower(col("location_cleaned")).contains("da nang") |
            lower(col("location_cleaned")).contains("danang"),
            lit("Da Nang")
        ).when(
            lower(col("location_cleaned")).contains("cần thơ") |
            lower(col("location_cleaned")).contains("can tho"),
            lit("Can Tho")
        ).when(
            lower(col("location_cleaned")).contains("hải phòng") |
            lower(col("location_cleaned")).contains("hai phong"),
            lit("Hai Phong")
        ).otherwise(lit("Other"))
    )
    
    return df_location

location_info_df = extract_location_info(location_analysis)

# Thống kê theo thành phố
print("\n===== THỐNG KÊ THEO THÀNH PHỐ =====")
city_stats = get_unique_values(location_info_df, "city_extracted")

# Thống kê theo quận (cho TPHCM)
hcm_data = location_info_df.filter(col("city_extracted") == "Ho Chi Minh")
if hcm_data.count() > 0:
    print("\n===== THỐNG KÊ QUẬN TPHCM =====")
    district_stats = get_unique_values(hcm_data, "district_extracted", 20)


===== PHÂN TÍCH VỊ TRÍ VÀ COORDINATES =====
Số bản ghi có coordinates hợp lệ: 17319
Tỷ lệ có coordinates: 99.9%

===== PHÂN TÍCH LOCATION TEXT =====
Độ dài location trung bình: 53.6 ký tự
Độ dài tối thiểu: 0
Độ dài tối đa: 109
Số location chi tiết (>10 ký tự): 17327

===== THỐNG KÊ THEO THÀNH PHỐ =====

===== Giá trị duy nhất của cột city_extracted =====
+--------------+-----+
|city_extracted|count|
+--------------+-----+
|Ho Chi Minh   |10009|
|Hanoi         |4177 |
|Other         |1555 |
|Da Nang       |1269 |
|Hai Phong     |255  |
|Can Tho       |71   |
+--------------+-----+


===== THỐNG KÊ QUẬN TPHCM =====

===== Giá trị duy nhất của cột district_extracted =====
+------------------+-----+
|district_extracted|count|
+------------------+-----+
|null              |4981 |
|1                 |1269 |
|3                 |750  |
|7                 |609  |
|10                |448  |
|2                 |384  |
|12                |360  |
|9                 |353  |
|6                 |276 

In [37]:
# Sửa lại hàm smart_fill_bedroom_bathroom với concat đúng cú pháp
from pyspark.sql.functions import (
    col, to_timestamp, current_timestamp, lit, regexp_replace, trim,
    when, upper, lower, split, element_at, round as spark_round,
    avg, count, percentile_approx, stddev, min as spark_min, max as spark_max,
    udf, length, expr, regexp_extract, concat, isnull, isnan
)

def smart_fill_bedroom_bathroom(df):
    """
    Điền bedroom/bathroom null dựa trên:
    1. Diện tích nhà
    2. Giá nhà  
    3. Thông tin từ title/description
    4. Statistical imputation dựa trên nhóm tương tự
    """
    
    # Đầu tiên, làm sạch các cột cần thiết
    df_cleaned = df.withColumn(
        "area_cleaned", 
        regexp_replace(col("area"), "[^0-9\\.]", "").cast("double")
    ).withColumn(
        "bedroom_cleaned", 
        regexp_replace(col("bedroom"), "[^0-9]", "").cast("double")
    ).withColumn(
        "bathroom_cleaned", 
        regexp_replace(col("bathroom"), "[^0-9]", "").cast("double")
    ).withColumn(
        "price_cleaned",
        when(
            lower(col("price")).contains("tỷ") | lower(col("price")).contains("ty"),
            regexp_replace(col("price"), "[^0-9\\.]", "").cast("double") * 1000000000
        ).when(
            lower(col("price")).contains("triệu") | lower(col("price")).contains("trieu"),
            regexp_replace(col("price"), "[^0-9\\.]", "").cast("double") * 1000000
        ).otherwise(
            regexp_replace(col("price"), "[^0-9\\.]", "").cast("double")
        )
    )
    
    # Trích xuất thành phố đơn giản
    df_with_city = df_cleaned.withColumn(
        "city_extracted",
        when(
            lower(col("location")).contains("hồ chí minh") |
            lower(col("location")).contains("tp.hcm") |
            lower(col("location")).contains("tphcm") |
            lower(col("location")).contains("hcm"),
            lit("Ho Chi Minh")
        ).when(
            lower(col("location")).contains("hà nội") |
            lower(col("location")).contains("hanoi"),
            lit("Hanoi")
        ).otherwise(lit("Other"))
    )
    
    # Bước 1: Trích xuất thông tin từ title và description
    print("Bước 1: Trích xuất từ title/description...")
    
    df_extracted = df_with_city.withColumn(
        "title_desc_combined",
        concat(
            when(col("title").isNotNull(), lower(col("title"))).otherwise(lit("")),
            lit(" "),
            when(col("description").isNotNull(), lower(col("description"))).otherwise(lit(""))
        )
    ).withColumn(
        "bedroom_from_text",
        when(
            col("title_desc_combined").rlike(r"(\d+)\s*(phòng\s*ngủ|pn|bedroom)"),
            regexp_extract(col("title_desc_combined"), r"(\d+)\s*(?:phòng\s*ngủ|pn|bedroom)", 1).cast("double")
        ).otherwise(lit(None))
    ).withColumn(
        "bathroom_from_text", 
        when(
            col("title_desc_combined").rlike(r"(\d+)\s*(phòng\s*tắm|wc|toilet|bathroom)"),
            regexp_extract(col("title_desc_combined"), r"(\d+)\s*(?:phòng\s*tắm|wc|toilet|bathroom)", 1).cast("double")
        ).otherwise(lit(None))
    )
    
    # Bước 2: Ước lượng dựa trên diện tích
    print("Bước 2: Ước lượng dựa trên diện tích...")
    
    df_area_based = df_extracted.withColumn(
        "bedroom_from_area",
        when(col("area_cleaned").isNotNull(),
            when(col("area_cleaned") <= 30, lit(1))
            .when(col("area_cleaned") <= 50, lit(2))
            .when(col("area_cleaned") <= 80, lit(3))
            .when(col("area_cleaned") <= 120, lit(4))
            .when(col("area_cleaned") <= 200, lit(5))
            .otherwise(lit(6))
        ).otherwise(lit(None))
    ).withColumn(
        "bathroom_from_area",
        when(col("area_cleaned").isNotNull(),
            when(col("area_cleaned") <= 40, lit(1))
            .when(col("area_cleaned") <= 80, lit(2))
            .when(col("area_cleaned") <= 150, lit(3))
            .otherwise(lit(4))
        ).otherwise(lit(None))
    )
    
    # Bước 3: Tính median theo nhóm location và price range
    print("Bước 3: Tính median theo nhóm tương tự...")
    
    # Tạo price range groups
    df_grouped = df_area_based.withColumn(
        "price_range",
        when(col("price_cleaned").isNull(), lit("unknown"))
        .when(col("price_cleaned") < 1000000000, lit("under_1b"))      # Dưới 1 tỷ
        .when(col("price_cleaned") < 3000000000, lit("1b_3b"))         # 1-3 tỷ
        .when(col("price_cleaned") < 5000000000, lit("3b_5b"))         # 3-5 tỷ
        .when(col("price_cleaned") < 10000000000, lit("5b_10b"))       # 5-10 tỷ
        .otherwise(lit("over_10b"))                                    # Trên 10 tỷ
    ).withColumn(
        "area_range",
        when(col("area_cleaned").isNull(), lit("unknown"))
        .when(col("area_cleaned") < 50, lit("small"))                  # Nhỏ < 50m²
        .when(col("area_cleaned") < 100, lit("medium"))                # Trung bình 50-100m²
        .when(col("area_cleaned") < 200, lit("large"))                 # Lớn 100-200m²
        .otherwise(lit("very_large"))                                  # Rất lớn > 200m²
    )
    
    # Tính median cho bedroom
    try:
        bedroom_median_by_group = df_grouped.filter(col("bedroom_cleaned").isNotNull()) \
            .select("city_extracted", "price_range", "area_range", "bedroom_cleaned") \
            .groupBy("city_extracted", "price_range", "area_range") \
            .agg(percentile_approx("bedroom_cleaned", 0.5).alias("bedroom_median"),
                 count("*").alias("bedroom_count")) \
            .filter(col("bedroom_count") >= 3)  # Chỉ lấy nhóm có ít nhất 3 samples
    except:
        # Fallback nếu không có đủ dữ liệu
        bedroom_median_by_group = spark.createDataFrame([], "city_extracted string, price_range string, area_range string, bedroom_median double, bedroom_count long")
    
    # Tính median cho bathroom  
    try:
        bathroom_median_by_group = df_grouped.filter(col("bathroom_cleaned").isNotNull()) \
            .select("city_extracted", "price_range", "area_range", "bathroom_cleaned") \
            .groupBy("city_extracted", "price_range", "area_range") \
            .agg(percentile_approx("bathroom_cleaned", 0.5).alias("bathroom_median"),
                 count("*").alias("bathroom_count")) \
            .filter(col("bathroom_count") >= 3)
    except:
        # Fallback nếu không có đủ dữ liệu
        bathroom_median_by_group = spark.createDataFrame([], "city_extracted string, price_range string, area_range string, bathroom_median double, bathroom_count long")
    
    # Join lại để có median values
    df_with_median = df_grouped \
        .join(bedroom_median_by_group, 
              ["city_extracted", "price_range", "area_range"], "left") \
        .join(bathroom_median_by_group, 
              ["city_extracted", "price_range", "area_range"], "left")
    
    # Bước 4: Fallback medians (toàn dataset)
    try:
        overall_bedroom_median = df_with_median.filter(col("bedroom_cleaned").isNotNull()) \
            .select(percentile_approx("bedroom_cleaned", 0.5)).collect()[0][0]
        overall_bathroom_median = df_with_median.filter(col("bathroom_cleaned").isNotNull()) \
            .select(percentile_approx("bathroom_cleaned", 0.5)).collect()[0][0]
    except:
        overall_bedroom_median = 2.0  # Default fallback
        overall_bathroom_median = 1.0  # Default fallback
    
    print(f"Overall bedroom median: {overall_bedroom_median}")
    print(f"Overall bathroom median: {overall_bathroom_median}")
    
    # Bước 5: Logic điền thông minh với độ ưu tiên
    df_smart_filled = df_with_median.withColumn(
        "bedroom_final",
        # Ưu tiên 1: Giá trị gốc nếu có
        when(col("bedroom_cleaned").isNotNull(), col("bedroom_cleaned"))
        # Ưu tiên 2: Trích xuất từ text
        .when(col("bedroom_from_text").isNotNull() & 
              (col("bedroom_from_text") >= 1) & (col("bedroom_from_text") <= 10), 
              col("bedroom_from_text"))
        # Ưu tiên 3: Median của nhóm tương tự
        .when(col("bedroom_median").isNotNull(), col("bedroom_median"))
        # Ưu tiên 4: Ước lượng từ diện tích
        .when(col("bedroom_from_area").isNotNull(), col("bedroom_from_area"))
        # Cuối cùng: Overall median
        .otherwise(lit(overall_bedroom_median))
    ).withColumn(
        "bathroom_final",
        # Ưu tiên 1: Giá trị gốc nếu có
        when(col("bathroom_cleaned").isNotNull(), col("bathroom_cleaned"))
        # Ưu tiên 2: Trích xuất từ text
        .when(col("bathroom_from_text").isNotNull() & 
              (col("bathroom_from_text") >= 1) & (col("bathroom_from_text") <= 5), 
              col("bathroom_from_text"))
        # Ưu tiên 3: Median của nhóm tương tự
        .when(col("bathroom_median").isNotNull(), col("bathroom_median"))
        # Ưu tiên 4: Ước lượng từ diện tích
        .when(col("bathroom_from_area").isNotNull(), col("bathroom_from_area"))
        # Cuối cùng: Overall median
        .otherwise(lit(overall_bathroom_median))
    )
    
    # Thêm flag để biết nguồn gốc của dữ liệu
    df_smart_filled = df_smart_filled.withColumn(
        "bedroom_source",
        when(col("bedroom_cleaned").isNotNull(), lit("original"))
        .when(col("bedroom_from_text").isNotNull() & 
              (col("bedroom_from_text") >= 1) & (col("bedroom_from_text") <= 10), 
              lit("extracted_from_text"))
        .when(col("bedroom_median").isNotNull(), lit("group_median"))
        .when(col("bedroom_from_area").isNotNull(), lit("area_based"))
        .otherwise(lit("overall_median"))
    ).withColumn(
        "bathroom_source",
        when(col("bathroom_cleaned").isNotNull(), lit("original"))
        .when(col("bathroom_from_text").isNotNull() & 
              (col("bathroom_from_text") >= 1) & (col("bathroom_from_text") <= 5), 
              lit("extracted_from_text"))
        .when(col("bathroom_median").isNotNull(), lit("group_median"))
        .when(col("bathroom_from_area").isNotNull(), lit("area_based"))
        .otherwise(lit("overall_median"))
    )
    
    return df_smart_filled

# Áp dụng smart filling
print("\n===== BẮT ĐẦU SMART FILLING =====")
smart_filled_df = smart_fill_bedroom_bathroom(df)

# Thống kê kết quả
print("\n===== THỐNG KÊ KẾT QUẢ SMART FILLING =====")

# So sánh trước và sau
original_bedroom_nulls = smart_filled_df.filter(col("bedroom_cleaned").isNull()).count()
original_bathroom_nulls = smart_filled_df.filter(col("bathroom_cleaned").isNull()).count()
total_records = smart_filled_df.count()

print(f"Tổng số bản ghi: {total_records}")
print(f"Bedroom null ban đầu: {original_bedroom_nulls} ({original_bedroom_nulls/total_records*100:.1f}%)")
print(f"Bathroom null ban đầu: {original_bathroom_nulls} ({original_bathroom_nulls/total_records*100:.1f}%)")

# Sau khi điền
final_bedroom_nulls = smart_filled_df.filter(col("bedroom_final").isNull()).count()
final_bathroom_nulls = smart_filled_df.filter(col("bathroom_final").isNull()).count()

print(f"Bedroom null sau điền: {final_bedroom_nulls} ({final_bedroom_nulls/total_records*100:.1f}%)")
print(f"Bathroom null sau điền: {final_bathroom_nulls} ({final_bathroom_nulls/total_records*100:.1f}%)")

# Thống kê nguồn gốc dữ liệu
print("\n===== NGUỒN GỐC DỮ LIỆU BEDROOM =====")
get_unique_values(smart_filled_df, "bedroom_source")

print("\n===== NGUỒN GỐC DỮ LIỆU BATHROOM =====")
get_unique_values(smart_filled_df, "bathroom_source")

# Kiểm tra độ hợp lý của kết quả
print("\n===== KIỂM TRA ĐỘ HỢP LÝ =====")
print("Phân bố bedroom_final:")
get_unique_values(smart_filled_df, "bedroom_final", 15)

print("Phân bố bathroom_final:")
get_unique_values(smart_filled_df, "bathroom_final", 10)


===== BẮT ĐẦU SMART FILLING =====
Bước 1: Trích xuất từ title/description...
Bước 2: Ước lượng dựa trên diện tích...
Bước 3: Tính median theo nhóm tương tự...
Overall bedroom median: 4.0
Overall bathroom median: 4.0

===== THỐNG KÊ KẾT QUẢ SMART FILLING =====
Tổng số bản ghi: 17336
Bedroom null ban đầu: 8343 (48.1%)
Bathroom null ban đầu: 8912 (51.4%)
Bedroom null sau điền: 0 (0.0%)
Bathroom null sau điền: 0 (0.0%)

===== NGUỒN GỐC DỮ LIỆU BEDROOM =====

===== Giá trị duy nhất của cột bedroom_source =====
+-------------------+-----+
|bedroom_source     |count|
+-------------------+-----+
|original           |8993 |
|group_median       |8090 |
|extracted_from_text|210  |
|area_based         |34   |
|overall_median     |9    |
+-------------------+-----+


===== NGUỒN GỐC DỮ LIỆU BATHROOM =====

===== Giá trị duy nhất của cột bathroom_source =====
+-------------------+-----+
|bathroom_source    |count|
+-------------------+-----+
|group_median       |8754 |
|original           |8424 |
|

DataFrame[bathroom_final: double, count: bigint]

In [39]:
# Sửa lại phần validation và tạo dataset cuối cùng
print("\n===== VALIDATION VÀ FINE-TUNING =====")

def validate_smart_filling(df):
    """Validation và điều chỉnh kết quả smart filling"""
    
    # Rule 1: Bathroom không được nhiều hơn bedroom + 1
    df_validated = df.withColumn(
        "bathroom_final_validated",
        when(
            col("bathroom_final") > (col("bedroom_final") + 1),
            col("bedroom_final")  # Điều chỉnh bathroom = bedroom nếu quá nhiều
        ).otherwise(col("bathroom_final"))
    )
    
    # Rule 2: Nhà nhỏ (< 30m²) không thể có quá nhiều phòng
    df_validated = df_validated.withColumn(
        "bedroom_final_validated",
        when(
            (col("area_cleaned") < 30) & (col("bedroom_final") > 2),
            lit(1)  # Nhà < 30m² tối đa 1-2 phòng ngủ
        ).when(
            (col("area_cleaned") < 50) & (col("bedroom_final") > 3),
            lit(2)  # Nhà < 50m² tối đa 2-3 phòng ngủ
        ).otherwise(col("bedroom_final"))
    )
    
    # Rule 3: Đảm bảo giá trị hợp lý
    df_validated = df_validated.withColumn(
        "bedroom_final_validated",
        when(col("bedroom_final_validated") < 1, lit(1))
        .when(col("bedroom_final_validated") > 10, lit(6))
        .otherwise(col("bedroom_final_validated"))
    ).withColumn(
        "bathroom_final_validated",
        when(col("bathroom_final_validated") < 1, lit(1))
        .when(col("bathroom_final_validated") > 6, lit(4))
        .otherwise(col("bathroom_final_validated"))
    )
    
    # Rule 4: Đánh dấu các trường hợp cần review
    df_validated = df_validated.withColumn(
        "needs_review",
        when(
            # Nhà rẻ (<2 tỷ) nhưng nhiều phòng (>4)
            (col("price_cleaned") < 2000000000) & (col("bedroom_final_validated") > 4),
            lit(True)
        ).when(
            # Diện tích và số phòng không khớp
            (col("area_cleaned") < 60) & (col("bedroom_final_validated") > 3),
            lit(True)
        ).otherwise(lit(False))
    )
    
    return df_validated

# Áp dụng validation
validated_df = validate_smart_filling(smart_filled_df)

# Thống kê các trường hợp cần review
print("\n===== CÁC TRƯỜNG HỢP CẦN REVIEW =====")
need_review = validated_df.filter(col("needs_review") == True)
print(f"Số bản ghi cần review: {need_review.count()}")

if need_review.count() > 0 and need_review.count() < 100:
    print("Mẫu các trường hợp cần review:")
    need_review.select(
        "area_cleaned", "price_cleaned", "bedroom_final_validated", 
        "bathroom_final_validated", "bedroom_source", "title"
    ).show(10, truncate=False)

# Tạo dataset cuối cùng - loại bỏ cột gốc trước khi tạo alias mới
final_room_df = validated_df.drop("bedroom", "bathroom").select(
    "*",
    col("bedroom_final_validated").alias("bedroom_new"),
    col("bathroom_final_validated").alias("bathroom_new")
)

print(f"\n===== DATASET CUỐI CÙNG =====")
print(f"Tổng số bản ghi: {final_room_df.count()}")
print(f"Bedroom null: {final_room_df.filter(col('bedroom_new').isNull()).count()}")
print(f"Bathroom null: {final_room_df.filter(col('bathroom_new').isNull()).count()}")

# So sánh với dataset ban đầu
original_valid = df.filter(
    col("bedroom").isNotNull() & col("bathroom").isNotNull()
).count()

final_valid = final_room_df.filter(
    col("bedroom_new").isNotNull() & col("bathroom_new").isNotNull()
).count()

print(f"Dataset ban đầu có bedroom+bathroom: {original_valid}")
print(f"Dataset sau smart filling: {final_valid}")
improvement = final_valid - original_valid
if original_valid > 0:
    improvement_pct = improvement / original_valid * 100
    print(f"Tăng thêm: {improvement} bản ghi ({improvement_pct:.1f}%)")

print("\n✅ HOÀN THÀNH SMART IMPUTATION CHO BEDROOM/BATHROOM")


===== VALIDATION VÀ FINE-TUNING =====

===== CÁC TRƯỜNG HỢP CẦN REVIEW =====
Số bản ghi cần review: 595

===== DATASET CUỐI CÙNG =====
Tổng số bản ghi: 17336
Bedroom null: 0
Bathroom null: 0
Dataset ban đầu có bedroom+bathroom: 17336
Dataset sau smart filling: 17336
Tăng thêm: 0 bản ghi (0.0%)

✅ HOÀN THÀNH SMART IMPUTATION CHO BEDROOM/BATHROOM


In [41]:
# Notebook xử lý tổng hợp dữ liệu bất động sản từ A-Z
print("🚀 BẮT ĐẦU QUY TRÌNH XỬ LÝ DỮ LIỆU BẤT ĐỘNG SAN")
print("=" * 70)

# Import đầy đủ
from pyspark.sql.functions import (
    col, to_timestamp, current_timestamp, lit, regexp_replace, trim,
    when, upper, lower, split, element_at, round as spark_round,
    avg, count, percentile_approx, stddev, min as spark_min, max as spark_max,
    udf, length, expr, regexp_extract, concat, isnull, isnan
)
from pyspark.sql.types import StringType, DoubleType

# =============================================================================
# BƯỚC 1: ĐỌC DỮ LIỆU
# =============================================================================
print("\n📂 BƯỚC 1: ĐỌC DỮ LIỆU")
csv_file = "/home/fer/data/real_estate_project/tmp/csv_files/bds_data_may2025.csv"
json_path = "hdfs://namenode:9000/data/realestate/raw/batdongsan/house/2025/05/*"

try:
    if os.path.exists(csv_file):
        df = spark.read.option("header", "true").csv(csv_file)
        print(f"✅ Đọc thành công từ CSV: {df.count()} bản ghi")
    else:
        df = spark.read.option("multiline", "false").json(json_path)
        print(f"✅ Đọc thành công từ JSON: {df.count()} bản ghi")
except Exception as e:
    print(f"❌ Lỗi đọc dữ liệu: {e}")
    # Tạo DataFrame trống với schema mẫu
    from pyspark.sql.types import StructType, StructField
    schema = StructType([
        StructField("url", StringType(), True),
        StructField("title", StringType(), True),
        StructField("price", StringType(), True),
        StructField("area", StringType(), True),
        StructField("bedroom", StringType(), True),
        StructField("bathroom", StringType(), True),
        StructField("house_direction", StringType(), True),
        StructField("interior", StringType(), True),
        StructField("legal_status", StringType(), True),
        StructField("location", StringType(), True),
        StructField("description", StringType(), True),
        StructField("posted_date", StringType(), True),
        StructField("crawl_timestamp", StringType(), True),
        StructField("latitude", StringType(), True),
        StructField("longitude", StringType(), True),
        StructField("seller_info", StringType(), True),
        StructField("source", StringType(), True),
        StructField("data_type", StringType(), True),
    ])
    df = spark.createDataFrame([], schema)

original_count = df.count()
print(f"📊 Dataset gốc: {original_count} bản ghi")

# =============================================================================
# BƯỚC 2: MAPPING VÀ CHUẨN HÓA DỮ LIỆU CATEGORICAL
# =============================================================================
print("\n🔄 BƯỚC 2: MAPPING VÀ CHUẨN HÓA DỮ LIỆU CATEGORICAL")

# 2.1. Mapping house_direction
print("   📍 Mapping house_direction...")
def normalize_direction(value):
    if value is None:
        return None
    return value.lower().replace(" ", "").replace("-", "")

direction_mapping = {
    # Hướng đơn
    "dong": "EAST", "đông": "EAST", "tay": "WEST", "tây": "WEST",
    "nam": "SOUTH", "bac": "NORTH", "bắc": "NORTH",
    # Hướng kép
    "dongnam": "SOUTHEAST", "đôngnam": "SOUTHEAST", "namdong": "SOUTHEAST", "namđông": "SOUTHEAST",
    "dongbac": "NORTHEAST", "đôngbắc": "NORTHEAST", "bacdong": "NORTHEAST", "bắcđông": "NORTHEAST",
    "taynam": "SOUTHWEST", "tâynam": "SOUTHWEST", "namtay": "SOUTHWEST", "namtây": "SOUTHWEST",
    "taybac": "NORTHWEST", "tâybắc": "NORTHWEST", "bactay": "NORTHWEST", "bắctây": "NORTHWEST",
    None: "UNKNOWN", "": "UNKNOWN"
}

normalize_direction_udf = udf(normalize_direction, StringType())
map_direction_udf = udf(lambda x: direction_mapping.get(x, "UNKNOWN"), StringType())

# 2.2. Mapping interior
print("   🏠 Mapping interior...")
def map_interior_by_keywords(value):
    if value is None or value == "":
        return "UNKNOWN"
    
    value_lower = value.lower()
    
    # LUXURY keywords
    luxury_keywords = ["caocấp", "cao cấp", "luxury", "sangtrọng", "sang trọng", "xịn", "5*", "5 sao",
                      "nhậpkhẩu", "nhập khẩu", "châuâu", "châu âu", "tiêuchuẩn", "tiêu chuẩn"]
    if any(keyword in value_lower for keyword in luxury_keywords):
        return "LUXURY"
    
    # FULLY_FURNISHED keywords
    fully_furnished_keywords = ["đầyđủ", "đầy đủ", "full", "hoànthiện", "hoàn thiện",
                               "trangbị", "trang bị", "điềuhòa", "điều hòa", "tủlạnh", "tủ lạnh",
                               "nộithất", "nội thất", "đểlại", "để lại", "tặng"]
    if any(keyword in value_lower for keyword in fully_furnished_keywords):
        return "FULLY_FURNISHED"
    
    # BASIC keywords
    basic_keywords = ["cơbản", "cơ bản", "bìnhthường", "bình thường", "chuẩn"]
    if any(keyword in value_lower for keyword in basic_keywords):
        return "BASIC"
    
    # UNFURNISHED keywords
    unfurnished_keywords = ["thô", "trống", "không", "k ", "nt", "nhàthô", "nhà thô"]
    if any(keyword in value_lower for keyword in unfurnished_keywords):
        return "UNFURNISHED"
    
    return "UNKNOWN"

map_interior_udf = udf(map_interior_by_keywords, StringType())

# 2.3. Mapping legal_status
print("   📋 Mapping legal_status...")
def map_legal_status_v2(value):
    if value is None or value == "":
        return "UNKNOWN"
    
    value_lower = (value.lower().replace(" ", "").replace("-", "").replace(".", "")
                  .replace("/", "").replace("\\", "").replace(",", "").replace(":", "")
                  .replace(";", "").replace("(", "").replace(")", "").replace("+", ""))
    
    # NO_LEGAL
    no_legal_keywords = ["khôngpháplý", "khongphapły", "khôngsổ", "khongso", "kosổ", "koso",
                        "không", "khong", "chưacó", "chuaco", "chưa", "chua", "ko"]
    if any(keyword in value_lower for keyword in no_legal_keywords):
        return "NO_LEGAL"

    # LAND_USE_CERTIFICATE
    land_use_keywords = ["thổcư", "thocu", "thổcư100", "thocu100", "thổcư100%", "thocu100%",
                        "đấtthổcư", "datthoju", "cnqsdđ", "cnqsdd", "sửdụngđất", "sudungdat"]
    if any(keyword in value_lower for keyword in land_use_keywords):
        return "LAND_USE_CERTIFICATE"

    # RED_BOOK
    red_book_keywords = ["sổđỏ", "sodo", "sổhồng", "sohong", "sổđỏsổhồng", "sđcc", "sdhh",
                        "bìađỏ", "biado", "sổchínhchủ", "sochinhchu", "sổđẹp", "sodep",
                        "sổvuông", "sovuong", "sổvuôngvắn", "sovuongvan", "sổsạch", "sosach"]
    if any(keyword in value_lower for keyword in red_book_keywords):
        return "RED_BOOK"

    # OWNERSHIP_CERTIFICATE
    ownership_keywords = ["shcc", "shr", "ccqsh", "chứngnhận", "chungnhan", "sổcc", "socc",
                         "côngcông", "congcong", "sổcôngnhận", "socognhan", "pháplý", "phapły",
                         "chínhchủ", "chinhchu", "sởhữu", "sohuu", "có", "co", "đầyđủ", "daydu"]
    if any(keyword in value_lower for keyword in ownership_keywords):
        return "OWNERSHIP_CERTIFICATE"

    # TRANSACTION_READY
    transaction_keywords = ["sẵnsànggiaodịch", "sansanggiaodich", "côngchứng", "congchung",
                           "giaodịchngay", "giaodichngay", "vuôngvắn", "vuongvan", "sạch", "sach"]
    if any(keyword in value_lower for keyword in transaction_keywords):
        return "TRANSACTION_READY"

    return "UNKNOWN"

map_legal_status_udf = udf(map_legal_status_v2, StringType())

# =============================================================================
# BƯỚC 3: SMART IMPUTATION CHO BEDROOM/BATHROOM
# =============================================================================
print("\n🧠 BƯỚC 3: SMART IMPUTATION CHO BEDROOM/BATHROOM")

def smart_fill_bedroom_bathroom(df):
    print("   🔍 Bước 3.1: Làm sạch dữ liệu số...")
    
    # Làm sạch các cột số
    df_cleaned = df.withColumn("area_cleaned", regexp_replace(col("area"), "[^0-9\\.]", "").cast("double")) \
                  .withColumn("bedroom_cleaned", regexp_replace(col("bedroom"), "[^0-9]", "").cast("double")) \
                  .withColumn("bathroom_cleaned", regexp_replace(col("bathroom"), "[^0-9]", "").cast("double")) \
                  .withColumn("price_cleaned",
                             when(lower(col("price")).contains("tỷ") | lower(col("price")).contains("ty"),
                                  regexp_replace(col("price"), "[^0-9\\.]", "").cast("double") * 1000000000)
                             .when(lower(col("price")).contains("triệu") | lower(col("price")).contains("trieu"),
                                   regexp_replace(col("price"), "[^0-9\\.]", "").cast("double") * 1000000)
                             .otherwise(regexp_replace(col("price"), "[^0-9\\.]", "").cast("double")))
    
    # Trích xuất thành phố
    df_with_city = df_cleaned.withColumn(
        "city_extracted",
        when(lower(col("location")).contains("hồ chí minh") | lower(col("location")).contains("tp.hcm") |
             lower(col("location")).contains("tphcm") | lower(col("location")).contains("hcm"), lit("Ho Chi Minh"))
        .when(lower(col("location")).contains("hà nội") | lower(col("location")).contains("hanoi"), lit("Hanoi"))
        .otherwise(lit("Other"))
    )
    
    print("   📝 Bước 3.2: Trích xuất từ title/description...")
    
    # Trích xuất thông tin từ text
    df_extracted = df_with_city.withColumn(
        "title_desc_combined",
        concat(when(col("title").isNotNull(), lower(col("title"))).otherwise(lit("")),
               lit(" "),
               when(col("description").isNotNull(), lower(col("description"))).otherwise(lit("")))
    ).withColumn(
        "bedroom_from_text",
        when(col("title_desc_combined").rlike(r"(\d+)\s*(phòng\s*ngủ|pn|bedroom)"),
             regexp_extract(col("title_desc_combined"), r"(\d+)\s*(?:phòng\s*ngủ|pn|bedroom)", 1).cast("double"))
        .otherwise(lit(None))
    ).withColumn(
        "bathroom_from_text", 
        when(col("title_desc_combined").rlike(r"(\d+)\s*(phòng\s*tắm|wc|toilet|bathroom)"),
             regexp_extract(col("title_desc_combined"), r"(\d+)\s*(?:phòng\s*tắm|wc|toilet|bathroom)", 1).cast("double"))
        .otherwise(lit(None))
    )
    
    print("   📏 Bước 3.3: Ước lượng từ diện tích...")
    
    # Ước lượng từ diện tích
    df_area_based = df_extracted.withColumn(
        "bedroom_from_area",
        when(col("area_cleaned").isNotNull(),
             when(col("area_cleaned") <= 30, lit(1))
             .when(col("area_cleaned") <= 50, lit(2))
             .when(col("area_cleaned") <= 80, lit(3))
             .when(col("area_cleaned") <= 120, lit(4))
             .when(col("area_cleaned") <= 200, lit(5))
             .otherwise(lit(6)))
        .otherwise(lit(None))
    ).withColumn(
        "bathroom_from_area",
        when(col("area_cleaned").isNotNull(),
             when(col("area_cleaned") <= 40, lit(1))
             .when(col("area_cleaned") <= 80, lit(2))
             .when(col("area_cleaned") <= 150, lit(3))
             .otherwise(lit(4)))
        .otherwise(lit(None))
    )
    
    print("   📊 Bước 3.4: Tính median theo nhóm...")
    
    # Tạo price và area ranges
    df_grouped = df_area_based.withColumn(
        "price_range",
        when(col("price_cleaned").isNull(), lit("unknown"))
        .when(col("price_cleaned") < 1000000000, lit("under_1b"))
        .when(col("price_cleaned") < 3000000000, lit("1b_3b"))
        .when(col("price_cleaned") < 5000000000, lit("3b_5b"))
        .when(col("price_cleaned") < 10000000000, lit("5b_10b"))
        .otherwise(lit("over_10b"))
    ).withColumn(
        "area_range",
        when(col("area_cleaned").isNull(), lit("unknown"))
        .when(col("area_cleaned") < 50, lit("small"))
        .when(col("area_cleaned") < 100, lit("medium"))
        .when(col("area_cleaned") < 200, lit("large"))
        .otherwise(lit("very_large"))
    )
    
    # Tính median cho bedroom và bathroom
    try:
        bedroom_median_by_group = df_grouped.filter(col("bedroom_cleaned").isNotNull()) \
            .groupBy("city_extracted", "price_range", "area_range") \
            .agg(percentile_approx("bedroom_cleaned", 0.5).alias("bedroom_median"),
                 count("*").alias("bedroom_count")) \
            .filter(col("bedroom_count") >= 3)
            
        bathroom_median_by_group = df_grouped.filter(col("bathroom_cleaned").isNotNull()) \
            .groupBy("city_extracted", "price_range", "area_range") \
            .agg(percentile_approx("bathroom_cleaned", 0.5).alias("bathroom_median"),
                 count("*").alias("bathroom_count")) \
            .filter(col("bathroom_count") >= 3)
    except:
        bedroom_median_by_group = spark.createDataFrame([], "city_extracted string, price_range string, area_range string, bedroom_median double, bedroom_count long")
        bathroom_median_by_group = spark.createDataFrame([], "city_extracted string, price_range string, area_range string, bathroom_median double, bathroom_count long")
    
    # Join median values
    df_with_median = df_grouped \
        .join(bedroom_median_by_group, ["city_extracted", "price_range", "area_range"], "left") \
        .join(bathroom_median_by_group, ["city_extracted", "price_range", "area_range"], "left")
    
    # Tính overall medians
    try:
        overall_bedroom_median = df_with_median.filter(col("bedroom_cleaned").isNotNull()) \
            .select(percentile_approx("bedroom_cleaned", 0.5)).collect()[0][0]
        overall_bathroom_median = df_with_median.filter(col("bathroom_cleaned").isNotNull()) \
            .select(percentile_approx("bathroom_cleaned", 0.5)).collect()[0][0]
    except:
        overall_bedroom_median = 2.0
        overall_bathroom_median = 1.0
    
    print(f"   📈 Overall medians - Bedroom: {overall_bedroom_median}, Bathroom: {overall_bathroom_median}")
    
    print("   🎯 Bước 3.5: Áp dụng logic điền thông minh...")
    
    # Logic điền thông minh theo độ ưu tiên
    df_smart_filled = df_with_median.withColumn(
        "bedroom_final",
        when(col("bedroom_cleaned").isNotNull(), col("bedroom_cleaned"))
        .when(col("bedroom_from_text").isNotNull() & 
              (col("bedroom_from_text") >= 1) & (col("bedroom_from_text") <= 10), 
              col("bedroom_from_text"))
        .when(col("bedroom_median").isNotNull(), col("bedroom_median"))
        .when(col("bedroom_from_area").isNotNull(), col("bedroom_from_area"))
        .otherwise(lit(overall_bedroom_median))
    ).withColumn(
        "bathroom_final",
        when(col("bathroom_cleaned").isNotNull(), col("bathroom_cleaned"))
        .when(col("bathroom_from_text").isNotNull() & 
              (col("bathroom_from_text") >= 1) & (col("bathroom_from_text") <= 5), 
              col("bathroom_from_text"))
        .when(col("bathroom_median").isNotNull(), col("bathroom_median"))
        .when(col("bathroom_from_area").isNotNull(), col("bathroom_from_area"))
        .otherwise(lit(overall_bathroom_median))
    )
    
    # Thêm source tracking
    df_smart_filled = df_smart_filled.withColumn(
        "bedroom_source",
        when(col("bedroom_cleaned").isNotNull(), lit("original"))
        .when(col("bedroom_from_text").isNotNull() & 
              (col("bedroom_from_text") >= 1) & (col("bedroom_from_text") <= 10), 
              lit("extracted_from_text"))
        .when(col("bedroom_median").isNotNull(), lit("group_median"))
        .when(col("bedroom_from_area").isNotNull(), lit("area_based"))
        .otherwise(lit("overall_median"))
    ).withColumn(
        "bathroom_source",
        when(col("bathroom_cleaned").isNotNull(), lit("original"))
        .when(col("bathroom_from_text").isNotNull() & 
              (col("bathroom_from_text") >= 1) & (col("bathroom_from_text") <= 5), 
              lit("extracted_from_text"))
        .when(col("bathroom_median").isNotNull(), lit("group_median"))
        .when(col("bathroom_from_area").isNotNull(), lit("area_based"))
        .otherwise(lit("overall_median"))
    )
    
    return df_smart_filled

# =============================================================================
# BƯỚC 4: ÁP DỤNG TẤT CẢ CÁC XỬ LÝ
# =============================================================================
print("\n⚙️ BƯỚC 4: ÁP DỤNG TẤT CẢ CÁC XỬ LÝ")

# Áp dụng tất cả mappings
processed_df = df.withColumn("house_direction_normalized", normalize_direction_udf(col("house_direction"))) \
                .withColumn("house_direction_mapped", map_direction_udf(col("house_direction_normalized"))) \
                .withColumn("interior_mapped", map_interior_udf(col("interior"))) \
                .withColumn("legal_status_mapped", map_legal_status_udf(col("legal_status")))

print("   ✅ Hoàn thành mapping categorical data")

# Áp dụng smart filling
smart_filled_df = smart_fill_bedroom_bathroom(processed_df)
print("   ✅ Hoàn thành smart imputation")

# =============================================================================
# BƯỚC 5: VALIDATION VÀ FINE-TUNING
# =============================================================================
print("\n🔍 BƯỚC 5: VALIDATION VÀ FINE-TUNING")

# Validation rules
validated_df = smart_filled_df.withColumn(
    "bathroom_final_validated",
    when(col("bathroom_final") > (col("bedroom_final") + 1), col("bedroom_final"))
    .otherwise(col("bathroom_final"))
).withColumn(
    "bedroom_final_validated",
    when((col("area_cleaned") < 30) & (col("bedroom_final") > 2), lit(1))
    .when((col("area_cleaned") < 50) & (col("bedroom_final") > 3), lit(2))
    .when(col("bedroom_final") < 1, lit(1))
    .when(col("bedroom_final") > 10, lit(6))
    .otherwise(col("bedroom_final"))
).withColumn(
    "bathroom_final_validated",
    when(col("bathroom_final_validated") < 1, lit(1))
    .when(col("bathroom_final_validated") > 6, lit(4))
    .otherwise(col("bathroom_final_validated"))
).withColumn(
    "needs_review",
    when((col("price_cleaned") < 2000000000) & (col("bedroom_final_validated") > 4), lit(True))
    .when((col("area_cleaned") < 60) & (col("bedroom_final_validated") > 3), lit(True))
    .otherwise(lit(False))
)

print("   ✅ Hoàn thành validation")

# =============================================================================
# BƯỚC 6: TẠO DATASET CUỐI CÙNG
# =============================================================================
print("\n📋 BƯỚC 6: TẠO DATASET CUỐI CÙNG")

# Làm sạch và chuẩn hóa thêm các cột khác
final_processed_df = validated_df.withColumn(
    "floor_count_cleaned", regexp_replace(col("floor_count"), "[^0-9]", "").cast("double")
).withColumn(
    "facade_width_cleaned", regexp_replace(col("facade_width"), "[^0-9\\.]", "").cast("double")
).withColumn(
    "road_width_cleaned", regexp_replace(col("road_width"), "[^0-9\\.]", "").cast("double")
).withColumn(
    "latitude_cleaned", col("latitude").cast("double")
).withColumn(
    "longitude_cleaned", col("longitude").cast("double")
).withColumn(
    "location_cleaned", trim(col("location"))
).withColumn(
    "is_negotiable", 
    when(lower(col("price")).contains("thỏa thuận") | 
         lower(col("price")).contains("thoathuan") | 
         lower(col("price")).contains("thoa thuan"), lit(True))
    .otherwise(lit(False))
).withColumn(
    "price_per_m2_cleaned",
    when(lower(col("price_per_m2")).contains("tỷ") | lower(col("price_per_m2")).contains("ty"),
         regexp_replace(col("price_per_m2"), "[^0-9\\.]", "").cast("double") * 1000000000)
    .when(lower(col("price_per_m2")).contains("triệu") | lower(col("price_per_m2")).contains("trieu"),
          regexp_replace(col("price_per_m2"), "[^0-9\\.]", "").cast("double") * 1000000)
    .otherwise(regexp_replace(col("price_per_m2"), "[^0-9\\.]", "").cast("double"))
).withColumn(
    "district_extracted",
    when(lower(col("location")).contains("quận"),
         regexp_extract(lower(col("location")), r"quận\s*(\d+)", 1))
    .when(lower(col("location")).contains("q."),
          regexp_extract(lower(col("location")), r"q\.?\s*(\d+)", 1))
    .otherwise(lit(None))
)

# Tạo dataset cuối cùng với cột đã đổi tên
final_clean_df = final_processed_df.drop("bedroom", "bathroom").select(
    # Thông tin cơ bản
    col("url"), col("title"), col("description"), col("posted_date"), col("crawl_timestamp"),
    col("seller_info"), col("source"), col("data_type"),
    
    # Thông tin nhà đất đã được làm sạch
    col("area_cleaned").alias("area"),
    col("bedroom_final_validated").alias("bedroom"),
    col("bathroom_final_validated").alias("bathroom"),
    col("floor_count_cleaned").alias("floor_count"),
    col("facade_width_cleaned").alias("facade_width"),
    col("road_width_cleaned").alias("road_width"),
    
    # Thông tin giá
    col("price_cleaned").alias("price"),
    col("price_per_m2_cleaned").alias("price_per_m2"),
    col("is_negotiable"),
    
    # Thông tin đã được mapped
    col("house_direction_mapped").alias("house_direction"),
    col("interior_mapped").alias("interior"),
    col("legal_status_mapped").alias("legal_status"),
    
    # Thông tin vị trí
    col("location_cleaned").alias("location"),
    col("latitude_cleaned").alias("latitude"),
    col("longitude_cleaned").alias("longitude"),
    col("city_extracted").alias("city"),
    col("district_extracted").alias("district"),
    
    # Metadata
    col("bedroom_source"), col("bathroom_source"), col("needs_review")
)

print("   ✅ Hoàn thành tạo dataset cuối cùng")

# =============================================================================
# BƯỚC 7: PHÂN TÍCH CHẤT LƯỢNG DỮ LIỆU
# =============================================================================
print("\n📊 BƯỚC 7: PHÂN TÍCH CHẤT LƯỢNG DỮ LIỆU")

# Tính điểm chất lượng
quality_df = final_clean_df.withColumn(
    "data_quality_score",
    (when(col("area").isNotNull() & (col("area") > 0), lit(20)).otherwise(lit(0))) +
    (when(col("price").isNotNull() & (col("price") > 0), lit(20)).otherwise(lit(0))) +
    (when(col("bedroom").isNotNull() & (col("bedroom") > 0), lit(15)).otherwise(lit(0))) +
    (when(col("bathroom").isNotNull() & (col("bathroom") > 0), lit(15)).otherwise(lit(0))) +
    (when(col("location").isNotNull() & (length(col("location")) > 10), lit(15)).otherwise(lit(0))) +
    (when(col("latitude").isNotNull() & col("longitude").isNotNull(), lit(15)).otherwise(lit(0)))
)

# Thống kê chất lượng
final_count = quality_df.count()
quality_stats = quality_df.select(
    avg("data_quality_score").alias("avg_quality"),
    spark_min("data_quality_score").alias("min_quality"),
    spark_max("data_quality_score").alias("max_quality"),
    count(when(col("data_quality_score") >= 80, True)).alias("high_quality_count"),
    count(when(col("data_quality_score") >= 60, True)).alias("medium_quality_count")
).collect()[0]

# Dataset theo chất lượng
high_quality_df = quality_df.filter(col("data_quality_score") >= 80)
medium_quality_df = quality_df.filter(col("data_quality_score") >= 60)

print("   ✅ Hoàn thành phân tích chất lượng")

# =============================================================================
# BƯỚC 8: THỐNG KÊ VÀ BÁO CÁO KẾT QUẢ
# =============================================================================
print("\n📈 BƯỚC 8: THỐNG KÊ VÀ BÁO CÁO KẾT QUẢ")
print("=" * 70)

# Thống kê tổng quan
print(f"📊 TỔNG QUAN:")
print(f"   • Dataset gốc: {original_count:,} bản ghi")
print(f"   • Dataset sau xử lý: {final_count:,} bản ghi")
print(f"   • Tỷ lệ giữ lại: {(final_count/original_count*100):.1f}%")

print(f"\n🎯 CHẤT LƯỢNG DỮ LIỆU:")
print(f"   • Điểm chất lượng trung bình: {quality_stats['avg_quality']:.1f}/100")
print(f"   • Dataset chất lượng cao (≥80): {quality_stats['high_quality_count']:,} ({quality_stats['high_quality_count']/final_count*100:.1f}%)")
print(f"   • Dataset chất lượng trung bình (≥60): {quality_stats['medium_quality_count']:,} ({quality_stats['medium_quality_count']/final_count*100:.1f}%)")

# Thống kê null values
print(f"\n🔍 NULL VALUES TRONG CÁC CỘT QUAN TRỌNG:")
important_cols = ["area", "bedroom", "bathroom", "price", "house_direction", "interior", "legal_status", "location"]
for col_name in important_cols:
    null_count = final_clean_df.filter(col(col_name).isNull()).count()
    null_pct = (null_count / final_count * 100) if final_count > 0 else 0
    print(f"   • {col_name}: {null_count:,} null ({null_pct:.1f}%)")

# Thống kê smart imputation
print(f"\n🧠 KẾT QUẢ SMART IMPUTATION:")
bedroom_sources = final_clean_df.groupBy("bedroom_source").count().orderBy("count", ascending=False).collect()
bathroom_sources = final_clean_df.groupBy("bathroom_source").count().orderBy("count", ascending=False).collect()

print("   📝 Nguồn gốc Bedroom:")
for row in bedroom_sources:
    pct = (row['count'] / final_count * 100) if final_count > 0 else 0
    print(f"     - {row['bedroom_source']}: {row['count']:,} ({pct:.1f}%)")

print("   🚿 Nguồn gốc Bathroom:")  
for row in bathroom_sources:
    pct = (row['count'] / final_count * 100) if final_count > 0 else 0
    print(f"     - {row['bathroom_source']}: {row['count']:,} ({pct:.1f}%)")

# Thống kê mapping results
print(f"\n🗺️ KẾT QUẢ MAPPING:")
print("   🧭 House Direction:")
direction_stats = final_clean_df.groupBy("house_direction").count().orderBy("count", ascending=False).limit(5).collect()
for row in direction_stats:
    pct = (row['count'] / final_count * 100) if final_count > 0 else 0
    print(f"     - {row['house_direction']}: {row['count']:,} ({pct:.1f}%)")

print("   🏠 Interior:")
interior_stats = final_clean_df.groupBy("interior").count().orderBy("count", ascending=False).limit(5).collect()
for row in interior_stats:
    pct = (row['count'] / final_count * 100) if final_count > 0 else 0
    print(f"     - {row['interior']}: {row['count']:,} ({pct:.1f}%)")

print("   📋 Legal Status:")
legal_stats = final_clean_df.groupBy("legal_status").count().orderBy("count", ascending=False).limit(5).collect()
for row in legal_stats:
    pct = (row['count'] / final_count * 100) if final_count > 0 else 0
    print(f"     - {row['legal_status']}: {row['count']:,} ({pct:.1f}%)")

# Thống kê theo thành phố
print(f"\n🌆 PHÂN BỐ THEO THÀNH PHỐ:")
city_stats = final_clean_df.groupBy("city").count().orderBy("count", ascending=False).collect()
for row in city_stats:
    pct = (row['count'] / final_count * 100) if final_count > 0 else 0
    print(f"   • {row['city']}: {row['count']:,} ({pct:.1f}%)")

# Phân tích giá (chỉ với dữ liệu có giá)
price_data = final_clean_df.filter(col("price").isNotNull() & (col("price") > 0))
if price_data.count() > 0:
    price_stats = price_data.select(
        avg("price").alias("avg_price"),
        percentile_approx("price", 0.5).alias("median_price"),
        spark_min("price").alias("min_price"),
        spark_max("price").alias("max_price")
    ).collect()[0]
    
    print(f"\n💰 THỐNG KÊ GIÁ ({price_data.count():,} bản ghi có giá):")
    print(f"   • Giá trung bình: {price_stats['avg_price']/1000000000:.2f} tỷ VNĐ")
    print(f"   • Giá trung vị: {price_stats['median_price']/1000000000:.2f} tỷ VNĐ")
    print(f"   • Giá thấp nhất: {price_stats['min_price']/1000000:.0f} triệu VNĐ")
    print(f"   • Giá cao nhất: {price_stats['max_price']/1000000000:.2f} tỷ VNĐ")

# Cần review
need_review_count = final_clean_df.filter(col("needs_review") == True).count()
print(f"\n⚠️ CẦN REVIEW: {need_review_count:,} bản ghi ({need_review_count/final_count*100:.1f}%)")

print("\n" + "=" * 70)
print("🎉 HOÀN THÀNH QUY TRÌNH XỬ LÝ DỮ LIỆU!")
print("✅ Các thành tựu chính:")
print("   • Mapping thành công house_direction, interior, legal_status")
print("   • Smart imputation cho bedroom/bathroom với nhiều chiến lược")
print("   • Làm sạch và chuẩn hóa dữ liệu số")
print("   • Trích xuất thông tin vị trí")
print("   • Phân loại chất lượng dữ liệu")
print("   • Validation và kiểm tra tính hợp lý")
print("=" * 70)

# Lưu kết quả (tùy chọn)
try:
    output_path = "/home/fer/data/real_estate_project/tmp/processed_data"
    os.makedirs(output_path, exist_ok=True)
    
    # Lưu dataset chất lượng cao
    high_quality_df.drop("data_quality_score").coalesce(1).write.mode("overwrite").parquet(f"{output_path}/high_quality_data.parquet")
    print(f"💾 Đã lưu {high_quality_df.count():,} bản ghi chất lượng cao vào: {output_path}/high_quality_data.parquet")
    
    # Lưu dataset đầy đủ
    quality_df.coalesce(1).write.mode("overwrite").parquet(f"{output_path}/full_processed_data.parquet")
    print(f"💾 Đã lưu {quality_df.count():,} bản ghi đầy đủ vào: {output_path}/full_processed_data.parquet")
    
except Exception as e:
    print(f"⚠️ Không thể lưu file: {e}")

print("\n🚀 Có thể sử dụng các DataFrame sau để phân tích tiếp:")
print("   • final_clean_df: Dataset đầy đủ đã xử lý")
print("   • high_quality_df: Dataset chất lượng cao (≥80 điểm)")
print("   • medium_quality_df: Dataset chất lượng trung bình (≥60 điểm)")

🚀 BẮT ĐẦU QUY TRÌNH XỬ LÝ DỮ LIỆU BẤT ĐỘNG SAN

📂 BƯỚC 1: ĐỌC DỮ LIỆU
✅ Đọc thành công từ JSON: 17336 bản ghi
📊 Dataset gốc: 17336 bản ghi

🔄 BƯỚC 2: MAPPING VÀ CHUẨN HÓA DỮ LIỆU CATEGORICAL
   📍 Mapping house_direction...
   🏠 Mapping interior...
   📋 Mapping legal_status...

🧠 BƯỚC 3: SMART IMPUTATION CHO BEDROOM/BATHROOM

⚙️ BƯỚC 4: ÁP DỤNG TẤT CẢ CÁC XỬ LÝ
   ✅ Hoàn thành mapping categorical data
   🔍 Bước 3.1: Làm sạch dữ liệu số...
   📝 Bước 3.2: Trích xuất từ title/description...
   📏 Bước 3.3: Ước lượng từ diện tích...
   📊 Bước 3.4: Tính median theo nhóm...
   📈 Overall medians - Bedroom: 4.0, Bathroom: 4.0
   🎯 Bước 3.5: Áp dụng logic điền thông minh...
   ✅ Hoàn thành smart imputation

🔍 BƯỚC 5: VALIDATION VÀ FINE-TUNING
   ✅ Hoàn thành validation

📋 BƯỚC 6: TẠO DATASET CUỐI CÙNG
   ✅ Hoàn thành tạo dataset cuối cùng

📊 BƯỚC 7: PHÂN TÍCH CHẤT LƯỢNG DỮ LIỆU
   ✅ Hoàn thành phân tích chất lượng

📈 BƯỚC 8: THỐNG KÊ VÀ BÁO CÁO KẾT QUẢ
📊 TỔNG QUAN:
   • Dataset gốc: 17,336 bản gh

25/05/24 21:11:08 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-223a686d-e2de-4535-9c91-de68178c4ec9. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-223a686d-e2de-4535-9c91-de68178c4ec9
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:177)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:113)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:94)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1231)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach